In [1]:
import json
import numpy as np

In [2]:
root_dir = 'all_annotations/dekra_annotations/'
annotations_dir = f'{root_dir}/annotations/instances_default.json'
image_dir = f'{root_dir}/images'

In [3]:

#with open(annotations_dir) as f:
#        configs = json.load(f)
        
#print(configs.keys())

In [4]:
#configs['categories']

In [5]:
from utility_functions.utilities_kie import * 
tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutxlm-base")
model = AutoModel.from_pretrained("microsoft/layoutxlm-base")

/home/a8hik/anaconda3/envs/py3_10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/layoutxlm-base were not used when initializing LayoutLMv2Model: ['layoutlmv2.visual.backbone.bottom_up.res4.20.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.0.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.3.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.0.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.1.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.2.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.4.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.0.conv1.norm.num_batches_tr

In [6]:
import pickle

def preprocess_tokens(tokens:list=None, bboxes:list=None):
    speical_characters = set(string.punctuation)
    tokens_preprocessed = []
    bboxes_preprocessed = []
        
    for i, token in enumerate(tokens):
        token = token.lower()
        for char_ in speical_characters:
            token = token.strip(char_)
        if token not in speical_characters and token not in (' ',  '', '\t','\n') and len(token) > 0:
            tokens_preprocessed.append(token)
            bboxes_preprocessed.append(bboxes[i])
        
    return tokens_preprocessed, bboxes_preprocessed

def load_annotation_image(image_id:int=None, configs:dict=None):
    annotations = []
    for annotation in configs["annotations"]:
        if annotation["image_id"] == image_id:
            annotations.append(annotation)
    return annotations
        
def load_category(category_id:int=None, configs:dict=None):
    for category in configs["categories"]:
        if category["id"] == category_id:
            return category


def postprocess_bobx(bbox, image_dim):
    bbox_postprocessed = []
    
    #for i in bbox:
    #    if i - float(int(i)) < 0.5:
    #        val = int(np.floor(i))
    #        bbox_postprocessed.append(val)
    #    else:
    #        val = int(np.ceil(i))
    #        bbox_postprocessed.append(val)
    bbox_postprocessed = [bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]]
    
    bbox_postprocessed = normalize_bbox(bbox_postprocessed, width=image_dim['width'], height = image_dim['height'])
    
    return bbox_postprocessed


def split_text_bboxes(bbox, text, category):
    #print(text)
    tokens = text.split(' ')
    bboxes = [bbox for _ in range(len(tokens))]
    tokens, bboxes = preprocess_tokens(tokens=tokens, bboxes=bboxes)
    categories = [category for _ in range(len(tokens))]
    return tokens, bboxes, categories

def form_entities(token_cat_groups, tokenizer, input_ids, input_id_map):
    entities = {'start':[], 'end':[], 'label':[], 'cat':[]}
    #print(token_cat_groups)
    for token_cat_group in token_cat_groups:
        token_group = token_cat_group[0]
        cat_group = token_cat_group[1]
        
        if len(cat_group) >0 and len(token_group) >0:
            cat = cat_group[0]
        
            input_ids_token_group = [input_id_map[tokenized_token] for token in token_group for tokenized_token in tokenizer.tokenize(token) if tokenized_token in input_id_map.keys()]
            indexes = find_sequnce_indices(sequence=input_ids_token_group, input_ids=input_ids)
        
            start_idx = indexes[0][0]
            end_idx = indexes[0][-1]
        
            entities['start'].append(start_idx)  
            entities['end'] .append(end_idx)
            if '_key' in cat:
                entities['label'].append('QUESTION')
            elif '_value' in cat:
                entities['label'].append('ANSWER')
            else:
                entities['label'].append('OTHER')
            entities['cat'].append(cat)
    return entities
    
        
        
           

def label_input_ids(annotation, category, bbox):
    label_vals = {'O' : 0, 'B-QUESTION' : 1, 'B-ANSWER' : 2, 'B-HEADER' : 3, 'I-ANSWER' : 4, 'I-QUESTION' : 5, 'I-HEADER' : 6}
    
    annotation_attributes = annotation['attributes']
    #bbox = postprocess_bobx(annotation['bbox'])
    text = annotation_attributes['content']
    tokens, _, _ = split_text_bboxes(bbox, text, category)
    labels = []
        
    for i in range(len(tokens)):
        if i==0:
            if 'key' in category:
                labels.append(label_vals['B-QUESTION'])
            elif 'value' in category:
                labels.append(label_vals['B-ANSWER'])
            else:
                labels.append(label_vals['O'])
        else:
            if 'key' in category:
                labels.append(label_vals['I-QUESTION'])
            elif 'value' in category:
                labels.append(label_vals['I-ANSWER'])
            else:
                labels.append(label_vals['O'])
        
    return labels

def form_relations(entities, input_ids):
    entity_names_cat = {'entity_names':[], 'cat':[]}
    relations={'head':[], 'tail':[]}
    for i , (start, end) in enumerate(zip(entities['start'], entities['end'])):
        #print(f'{i}, {tokenizer.decode(input_ids[start:end])} : {entities["label"][i]} {entities["cat"][i]}')
        #if entities["label"][i].lower() != 'other':
        entity_names_cat['entity_names'].append(tokenizer.decode(input_ids[start:end]))
        entity_names_cat['cat'].append(entities['cat'][i])
 
    for i, entity in enumerate(entity_names_cat['entity_names']):
        cat = entity_names_cat['cat'][i]
        if '_key' in cat.lower():         
            j = entity_names_cat['cat'].index(f'{cat[:-4:]}_value') if f'{cat[:-4:]}_value' in entity_names_cat['cat'] else None
            if j is not None:
                head = i
                tail = j
                relations['head'].append(head)
                relations['tail'].append(tail)
            
                
            
        
    return entity_names_cat, relations
        
def dump_pickle(file_name:str=None, collection:object=None):
        file_name = os.path.join(os.getcwd(), file_name)
        with open(file_name, 'wb') as f:
            pickle.dump(collection, f)
            
def combine_annotations(image_id:int =None, configs=None, tokenizer=None, model =None, image_dim:dict=None):
    annotations = load_annotation_image(image_id=image_id, configs=configs)
    filtered_annotations = {'image_id':image_id, 'bboxes':[], 'categories':[], 'text':[], 'labels' :[], 'input_ids':[], 'entities':[], 'relations':[], 'input_id_map':{}, 'token_cat_groups':[]}
    for annotation in annotations:
        bbox = postprocess_bobx(annotation['bbox'], image_dim)
        category = load_category(category_id=annotation['category_id'], configs=configs)["name"]
        
        annotation_attributes = annotation['attributes']
        if 'type' in annotation_attributes.keys():
            type = annotation_attributes['type']
            filtered_annotations['categories'].append(f'{category}_{type}')
            category = f'{category}_{type}'
        
        text = annotation_attributes['content']
        tokens, bboxes, categories = split_text_bboxes(bbox, text, category)
        token_cat_group = (tokens, categories)
        filtered_annotations['token_cat_groups'].append(token_cat_group)
        labels = label_input_ids(annotation, category, bbox)
        filtered_annotations['bboxes'].extend(bboxes)
        filtered_annotations['text'].extend(tokens)
        filtered_annotations['categories'].extend(categories)
        filtered_annotations['labels'].extend(labels)
    
    tokens, bbox = preprocess_tokens(tokens=filtered_annotations['text'], bboxes=filtered_annotations['bboxes'])
    tokenizer, model = add_tokens_tokenizer(tokens = tokens, tokenizer = tokenizer, model = model)
    input_ids, bboxes, input_id_map = encode_tokens(tokens=tokens, bboxes=bbox, tokenizer=tokenizer)
    filtered_annotations['input_id_map'] = input_id_map
    filtered_annotations['input_ids'].extend(input_ids)
    
    #print(input_ids)
    #print(input_id_map)
    
        
    return filtered_annotations

        

In [7]:
images_dir = "images"
images_resized_dir = "images_resized"
bbox_dir="bbox"
input_ids_dir = "input_ids"
labels_dir = "labels"
entities_dir = "entities"
relations_dir = "relations"

## Impl 1 dataset generation funsd

In [8]:
from tqdm import tqdm
import random
import shutil

'''def create_directory_structure(dataset_dir:str, sub_dir:list, overwrite:bool):
    if overwrite and os.path.exists(dataset_dir):
        shutil.rmtree(dataset_dir)
        
    os.mkdir(dataset_dir)
    
    sub_dirs=['train', 'validation']
    for sub_dir in sub_dirs:
        inner_dir =f'{dataset_dir}/{sub_dir}' 
        os.mkdir(inner_dir)
        os.mkdir(f'{inner_dir}/{images_dir}')
        os.mkdir(f'{inner_dir}/{images_resized_dir}')
        os.mkdir(f'{inner_dir}/{bbox_dir}')
        os.mkdir(f'{inner_dir}/{labels_dir}')
        os.mkdir(f'{inner_dir}/{entities_dir}')
        os.mkdir(f'{inner_dir}/{relations_dir}')
        os.mkdir(f'{inner_dir}/{input_ids_dir}')

def load_dataset_funsd(annotations_config:str=None, image_dir:str=None, dataset_dir:str= 'dataset_real', tokenizer=None, model=None, train_test_split:float=0.05, overwrite :bool=True):
    sub_dirs = ['train', 'validation']
    create_directory_structure(dataset_dir, sub_dirs, True)
 
    with open(annotations_config) as f:
        configs = json.load(f)
    label_vals = {'O' : 0, 'B-QUESTION' : 1, 'B-ANSWER' : 2, 'B-HEADER' : 3, 'I-ANSWER' : 4, 'I-QUESTION' : 5, 'I-HEADER' : 6}
    label_vals_reversed = {val:key for (key,val) in label_vals.items()} 
    image_ids = []    
    indices_to_keep = []
    print('populating valid image indices')
    for i,image_obj in enumerate(configs['images']):
        if i>=0:
            image_ids.append(image_obj['id'])
            
            image_path = f'{image_dir}/{image_obj["file_name"]}'
            img_height = image_obj["height"]
            img_width = image_obj["width"]
        
            combined_annotations = combine_annotations(image_id=image_obj['id'], configs=configs, tokenizer=tokenizer, model=model, image_dim={'width':img_width, 'height':img_height})
            entities = form_entities(combined_annotations['token_cat_groups'], tokenizer, combined_annotations['input_ids'], combined_annotations['input_id_map'])
            combined_annotations['entities'] = entities
            entity_names_cat, relations = form_relations(entities, combined_annotations['input_ids'])
            if len(relations['head']) > 0:
                indices_to_keep.append(i)
            print(f'{i} {image_obj["file_name"]}')
            for i in range(len(relations['head'])):
                print(f'question : {entity_names_cat["entity_names"][relations["head"][i]]}, Answer :  {entity_names_cat["entity_names"][relations["tail"][i]]}, start: {relations["head"][i]}, end: {relations["tail"][i]}')
            print('------------')
            #for i in range(len(relations['head'])):
            #    print(f'question : {entity_names_cat["entity_names"][relations["head"][i]]}, Answer :  {entity_names_cat["entity_names"][relations["tail"][i]]}, start: {relations["head"][i]}, end: {relations["tail"][i]}')
        
            #print('-------------------------------------------------')
    print(indices_to_keep)
    print('Generating dataset')
    validation_percentage = int(train_test_split * len(indices_to_keep))
    
    validation_indices = random.choices(indices_to_keep, k= validation_percentage)
    for i,image_obj in enumerate(configs['images']):
        if i in indices_to_keep:
            image_ids.append(image_obj['id'])    
            image_path = f'{image_dir}/{image_obj["file_name"]}'
            img_height = image_obj["height"]
            img_width = image_obj["width"]
        
            combined_annotations = combine_annotations(image_id=image_obj['id'], configs=configs, tokenizer=tokenizer, model=model, image_dim={'width':img_width, 'height':img_height})
            entities = form_entities(combined_annotations['token_cat_groups'], tokenizer, combined_annotations['input_ids'], combined_annotations['input_id_map'])
            combined_annotations['entities'] = entities
            entity_names_cat, relations = form_relations(entities, combined_annotations['input_ids'])
            
            image = cv2.imread(image_path)
            image_resized = cv2.resize(image, (224,224))
            bbox = combined_annotations['bboxes']
            input_ids = combined_annotations['input_ids']
            labels = combined_annotations['labels']
        
            if i not in validation_indices:
                cv2.imwrite(f'{dataset_dir}/train/{images_dir}/image_{indices_to_keep.index(i)}.jpeg', image)
                cv2.imwrite(f'{dataset_dir}/train/{images_resized_dir}/image_resized_{indices_to_keep.index(i)}.jpeg', image_resized)
                bbox_path = f'{dataset_dir}/train/{bbox_dir}/bbox_{indices_to_keep.index(i)}.p'
                labels_path = f'{dataset_dir}/train/{labels_dir}/labels_{indices_to_keep.index(i)}.p'
                entities_path = f'{dataset_dir}/train/{entities_dir}/entities_{indices_to_keep.index(i)}.p'
                input_ids_path = f'{dataset_dir}/train/{input_ids_dir}/input_ids_{indices_to_keep.index(i)}.p'
                realations_path = f'{dataset_dir}/train/{relations_dir}/relations_{indices_to_keep.index(i)}.p'
            
                dump_pickle(file_name=bbox_path, collection=bbox)
                dump_pickle(file_name=labels_path, collection=labels)
                dump_pickle(file_name=entities_path, collection=entities)
                dump_pickle(file_name=input_ids_path, collection=input_ids)
                dump_pickle(file_name=realations_path, collection=relations)
            else:
                cv2.imwrite(f'{dataset_dir}/validation/{images_dir}/image_{indices_to_keep.index(i)}.jpeg', image)
                cv2.imwrite(f'{dataset_dir}/validation/{images_resized_dir}/image_resized_{indices_to_keep.index(i)}.jpeg', image_resized)
                bbox_path = f'{dataset_dir}/validation/{bbox_dir}/bbox_{indices_to_keep.index(i)}.p'
                labels_path = f'{dataset_dir}/validation/{labels_dir}/labels_{indices_to_keep.index(i)}.p'
                entities_path = f'{dataset_dir}/validation/{entities_dir}/entities_{indices_to_keep.index(i)}.p'
                input_ids_path = f'{dataset_dir}/validation/{input_ids_dir}/input_ids_{indices_to_keep.index(i)}.p'
                realations_path = f'{dataset_dir}/validation/{relations_dir}/relations_{indices_to_keep.index(i)}.p'
            
                dump_pickle(file_name=bbox_path, collection=bbox)
                dump_pickle(file_name=labels_path, collection=labels)
                dump_pickle(file_name=entities_path, collection=entities)
                dump_pickle(file_name=input_ids_path, collection=input_ids)
                dump_pickle(file_name=realations_path, collection=relations)'''
            
            
            
            
        
         
            
        

'def create_directory_structure(dataset_dir:str, sub_dir:list, overwrite:bool):\n    if overwrite and os.path.exists(dataset_dir):\n        shutil.rmtree(dataset_dir)\n        \n    os.mkdir(dataset_dir)\n    \n    sub_dirs=[\'train\', \'validation\']\n    for sub_dir in sub_dirs:\n        inner_dir =f\'{dataset_dir}/{sub_dir}\' \n        os.mkdir(inner_dir)\n        os.mkdir(f\'{inner_dir}/{images_dir}\')\n        os.mkdir(f\'{inner_dir}/{images_resized_dir}\')\n        os.mkdir(f\'{inner_dir}/{bbox_dir}\')\n        os.mkdir(f\'{inner_dir}/{labels_dir}\')\n        os.mkdir(f\'{inner_dir}/{entities_dir}\')\n        os.mkdir(f\'{inner_dir}/{relations_dir}\')\n        os.mkdir(f\'{inner_dir}/{input_ids_dir}\')\n\ndef load_dataset_funsd(annotations_config:str=None, image_dir:str=None, dataset_dir:str= \'dataset_real\', tokenizer=None, model=None, train_test_split:float=0.05, overwrite :bool=True):\n    sub_dirs = [\'train\', \'validation\']\n    create_directory_structure(dataset_dir, sub

## Impl 2 dataset generation funsd

In [20]:
from tqdm import tqdm
import random
import shutil

def create_directory_structure(dataset_dir:str, sub_dir:list, overwrite:bool):
    if overwrite and os.path.exists(dataset_dir):
        shutil.rmtree(dataset_dir)
        
    os.mkdir(dataset_dir)
    
    sub_dirs=['train', 'validation']
    for sub_dir in sub_dirs:
        inner_dir =f'{dataset_dir}/{sub_dir}' 
        os.mkdir(inner_dir)
        os.mkdir(f'{inner_dir}/{images_dir}')
        os.mkdir(f'{inner_dir}/{images_resized_dir}')
        os.mkdir(f'{inner_dir}/{bbox_dir}')
        os.mkdir(f'{inner_dir}/{labels_dir}')
        os.mkdir(f'{inner_dir}/{entities_dir}')
        os.mkdir(f'{inner_dir}/{relations_dir}')
        os.mkdir(f'{inner_dir}/{input_ids_dir}')

def get_file_index_map(path:str=None):
        paths = os.listdir(path)
        sorted_paths = sorted(paths)
        idx_map = {i:sorted_paths[i] for i in range(len(sorted_paths))}
        return idx_map

def load_dataset_funsd(annotations_configs:dict=None, image_dirs:dict=None, dataset_dir:str= 'dataset_real', tokenizer=None, model=None, train_test_split:float=0.05, overwrite :bool=True, dataset='all'):
    sub_dirs = ['train', 'validation']
    create_directory_structure(dataset_dir, sub_dirs, True)
    configs_list = []
    image_dirs_list = []
    if dataset=='all':
        for key in annotations_configs.keys():
            configs_list.append(annotations_configs[key])
            image_dirs_list.append(image_dirs[key])
    else: 
        configs_list.append(annotations_configs[dataset])
        image_dirs_list.append(image_dirs[dataset])
        
    count = 0
    for annotations_config, image_dir in zip(configs_list, image_dirs_list):
        record_count = 0
        with open(annotations_config) as f:
            configs = json.load(f)
        label_vals = {'O' : 0, 'B-QUESTION' : 1, 'B-ANSWER' : 2, 'B-HEADER' : 3, 'I-ANSWER' : 4, 'I-QUESTION' : 5, 'I-HEADER' : 6}
        label_vals_reversed = {val:key for (key,val) in label_vals.items()} 
        image_ids = []    
        indices_to_keep = []
        print('populating valid image indices')
        for i,image_obj in enumerate(configs['images']):
            if i>=0:
                image_ids.append(image_obj['id'])
            
                image_path = f'{image_dir}/{image_obj["file_name"]}'
                img_height = image_obj["height"]
                img_width = image_obj["width"]
        
                combined_annotations = combine_annotations(image_id=image_obj['id'], configs=configs, tokenizer=tokenizer, model=model, image_dim={'width':img_width, 'height':img_height})
                entities = form_entities(combined_annotations['token_cat_groups'], tokenizer, combined_annotations['input_ids'], combined_annotations['input_id_map'])
                combined_annotations['entities'] = entities
                entity_names_cat, relations = form_relations(entities, combined_annotations['input_ids'])
                if len(relations['head']) > 0:
                    #indices_to_keep.append(i)
                    #print(f'{i} {image_obj["file_name"]}')
                    #for i in range(len(relations['head'])):
                    #    print(f'question : {entity_names_cat["entity_names"][relations["head"][i]]}, Answer :  {entity_names_cat["entity_names"][relations["tail"][i]]}, start: {relations["head"][i]}, end: {relations["tail"][i]}')
                    #print('------------')
                    image = cv2.imread(image_path)
                    image_resized = cv2.resize(image, (224,224))
                    bbox = combined_annotations['bboxes']
                    input_ids = combined_annotations['input_ids']
                    labels = combined_annotations['labels']
                
                    cv2.imwrite(f'{dataset_dir}/train/{images_dir}/image_{i+count}.jpeg', image)
                    cv2.imwrite(f'{dataset_dir}/train/{images_resized_dir}/image_resized_{i+count}.jpeg', image_resized)
                    bbox_path = f'{dataset_dir}/train/{bbox_dir}/bbox_{i+count}.p'
                    labels_path = f'{dataset_dir}/train/{labels_dir}/labels_{i+count}.p'
                    entities_path = f'{dataset_dir}/train/{entities_dir}/entities_{i+count}.p'
                    input_ids_path = f'{dataset_dir}/train/{input_ids_dir}/input_ids_{i+count}.p'
                    realations_path = f'{dataset_dir}/train/{relations_dir}/relations_{i+count}.p'
            
                    dump_pickle(file_name=bbox_path, collection=bbox)
                    dump_pickle(file_name=labels_path, collection=labels)
                    dump_pickle(file_name=entities_path, collection=entities)
                    dump_pickle(file_name=input_ids_path, collection=input_ids)
                    dump_pickle(file_name=realations_path, collection=relations)
                    record_count+=1
        count+=record_count

def split_data(train_test_split:float=None, dataset_dir:str=None):
    file_indices = [i for i in range(len(os.listdir(f'{dataset_dir}/train/images')))]
    validation_percentage = int(train_test_split * len(file_indices))
    validation_indices = random.choices(file_indices, k= validation_percentage)
    
    idx_map_image = get_file_index_map(path = f'{dataset_dir}/train/images')
    idx_map_image_resized = get_file_index_map(path = f'{dataset_dir}/train/images_resized')
    idx_map_bbox = get_file_index_map(path = f'{dataset_dir}/train/bbox')
    idx_map_labels = get_file_index_map(path = f'{dataset_dir}/train/labels')
    idx_map_entities = get_file_index_map(path = f'{dataset_dir}/train/entities')
    idx_map_realations = get_file_index_map(path = f'{dataset_dir}/train/relations')
    idx_map_input_ids = get_file_index_map(path = f'{dataset_dir}/train/input_ids')
    
    print(validation_indices)
    for i in validation_indices:
        shutil.move(f'{dataset_dir}/train/images/{idx_map_image[i]}', f'{dataset_dir}/validation/images/{idx_map_image[i]}')
        shutil.move(f'{dataset_dir}/train/images_resized/{idx_map_image_resized[i]}', f'{dataset_dir}/validation/images_resized/{idx_map_image_resized[i]}')
        shutil.move(f'{dataset_dir}/train/bbox/{idx_map_bbox[i]}', f'{dataset_dir}/validation/bbox/{idx_map_bbox[i]}')
        shutil.move(f'{dataset_dir}/train/input_ids/{idx_map_input_ids[i]}', f'{dataset_dir}/validation/input_ids/{idx_map_input_ids[i]}')
        shutil.move(f'{dataset_dir}/train/labels/{idx_map_labels[i]}', f'{dataset_dir}/validation/labels/{idx_map_labels[i]}')
        shutil.move(f'{dataset_dir}/train/entities/{idx_map_entities[i]}', f'{dataset_dir}/validation/entities/{idx_map_entities[i]}')
        shutil.move(f'{dataset_dir}/train/relations/{idx_map_realations[i]}', f'{dataset_dir}/validation/relations/{idx_map_realations[i]}')
    

## Impl3 dataset generation funsd

In [10]:
from tqdm import tqdm
import random
import shutil

'''def create_directory_structure(dataset_dir:str, sub_dir:list, overwrite:bool):
    if overwrite and os.path.exists(dataset_dir):
        shutil.rmtree(dataset_dir)
        
    os.mkdir(dataset_dir)
    
    sub_dirs=['train', 'validation']
    for sub_dir in sub_dirs:
        inner_dir =f'{dataset_dir}/{sub_dir}' 
        os.mkdir(inner_dir)
        os.mkdir(f'{inner_dir}/{images_dir}')
        os.mkdir(f'{inner_dir}/{images_resized_dir}')
        os.mkdir(f'{inner_dir}/{bbox_dir}')
        os.mkdir(f'{inner_dir}/{labels_dir}')
        os.mkdir(f'{inner_dir}/{entities_dir}')
        os.mkdir(f'{inner_dir}/{relations_dir}')
        os.mkdir(f'{inner_dir}/{input_ids_dir}')

def load_dataset_funsd(annotations_configs:dict=None, image_dirs:dict=None, dataset_dir:str= 'dataset_real', tokenizer=None, model=None, train_test_split:float=0.05, overwrite :bool=True, dataset='all'):
    sub_dirs = ['train', 'validation']
    create_directory_structure(dataset_dir, sub_dirs, True)
    
    configs_list = []
    image_dirs_list = []
    if dataset=='all':
        for key in annotations_configs.keys():
            configs_list.append(annotations_configs[key])
            image_dirs_list.append(image_dirs[key])
    else: 
        configs_list.append(annotations_configs[dataset])
        image_dirs_list.append(image_dirs[dataset])
    
    print(image_dirs_list)
    print(configs_list)
    count = 0
    for image_dir,annotations_config in zip(image_dirs_list, configs_list):
        with open(annotations_config) as f:
            configs = json.load(f)
        label_vals = {'O' : 0, 'B-QUESTION' : 1, 'B-ANSWER' : 2, 'B-HEADER' : 3, 'I-ANSWER' : 4, 'I-QUESTION' : 5, 'I-HEADER' : 6}
        label_vals_reversed = {val:key for (key,val) in label_vals.items()} 
        image_ids = []    
        indices_to_keep = []
        print(f'populating valid image indices for {annotations_config}')
        for i,image_obj in enumerate(configs['images']):
            image_ids.append(image_obj['id'])
            
            image_path = f'{image_dir}/{image_obj["file_name"]}'
            img_height = image_obj["height"]
            img_width = image_obj["width"]
        
            combined_annotations = combine_annotations(image_id=image_obj['id'], configs=configs, tokenizer=tokenizer, model=model, image_dim={'width':img_width, 'height':img_height})
            entities = form_entities(combined_annotations['token_cat_groups'], tokenizer, combined_annotations['input_ids'], combined_annotations['input_id_map'])
            combined_annotations['entities'] = entities
            _, relations = form_relations(entities, combined_annotations['input_ids'])
            if len(relations['head']) > 0:
                indices_to_keep.append(i)
            
        
        print(f'Generating dataset {annotations_config}')
        validation_percentage = int(train_test_split * len(indices_to_keep))
    
        validation_indices = random.choices(indices_to_keep, k= validation_percentage)
        for i,image_obj in enumerate(configs['images']):
            if i in indices_to_keep:
                image_ids.append(image_obj['id'])    
                image_path = f'{image_dir}/{image_obj["file_name"]}'
                img_height = image_obj["height"]
                img_width = image_obj["width"]
        
                combined_annotations = combine_annotations(image_id=image_obj['id'], configs=configs, tokenizer=tokenizer, model=model, image_dim={'width':img_width, 'height':img_height})
                entities = form_entities(combined_annotations['token_cat_groups'], tokenizer, combined_annotations['input_ids'], combined_annotations['input_id_map'])
                combined_annotations['entities'] = entities
                _, relations = form_relations(entities, combined_annotations['input_ids'])

                image = cv2.imread(image_path)
                image_resized = cv2.resize(image, (224,224))
                bbox = combined_annotations['bboxes']
                input_ids = combined_annotations['input_ids']
                labels = combined_annotations['labels']
        
                if i not in validation_indices:
                    cv2.imwrite(f'{dataset_dir}/train/{images_dir}/image_{indices_to_keep.index(i) + count}.jpeg', image)
                    cv2.imwrite(f'{dataset_dir}/train/{images_resized_dir}/image_resized_{indices_to_keep.index(i)+count}.jpeg', image_resized)
                    bbox_path = f'{dataset_dir}/train/{bbox_dir}/bbox_{indices_to_keep.index(i)+count}.p'
                    labels_path = f'{dataset_dir}/train/{labels_dir}/labels_{indices_to_keep.index(i)+count}.p'
                    entities_path = f'{dataset_dir}/train/{entities_dir}/entities_{indices_to_keep.index(i)+count}.p'
                    input_ids_path = f'{dataset_dir}/train/{input_ids_dir}/input_ids_{indices_to_keep.index(i)+count}.p'
                    realations_path = f'{dataset_dir}/train/{relations_dir}/relations_{indices_to_keep.index(i)+count}.p'
            
                    dump_pickle(file_name=bbox_path, collection=bbox)
                    dump_pickle(file_name=labels_path, collection=labels)
                    dump_pickle(file_name=entities_path, collection=entities)
                    dump_pickle(file_name=input_ids_path, collection=input_ids)
                    dump_pickle(file_name=realations_path, collection=relations)
                else:
                    cv2.imwrite(f'{dataset_dir}/validation/{images_dir}/image_{indices_to_keep.index(i)+count}.jpeg', image)
                    cv2.imwrite(f'{dataset_dir}/validation/{images_resized_dir}/image_resized_{indices_to_keep.index(i)+count}.jpeg', image_resized)
                    bbox_path = f'{dataset_dir}/validation/{bbox_dir}/bbox_{indices_to_keep.index(i)+count}.p'
                    labels_path = f'{dataset_dir}/validation/{labels_dir}/labels_{indices_to_keep.index(i)+count}.p'
                    entities_path = f'{dataset_dir}/validation/{entities_dir}/entities_{indices_to_keep.index(i)+count}.p'
                    input_ids_path = f'{dataset_dir}/validation/{input_ids_dir}/input_ids_{indices_to_keep.index(i)+count}.p'
                    realations_path = f'{dataset_dir}/validation/{relations_dir}/relations_{indices_to_keep.index(i)+count}.p'
            
                    dump_pickle(file_name=bbox_path, collection=bbox)
                    dump_pickle(file_name=labels_path, collection=labels)
                    dump_pickle(file_name=entities_path, collection=entities)
                    dump_pickle(file_name=input_ids_path, collection=input_ids)
                    dump_pickle(file_name=realations_path, collection=relations)
        count += len(indices_to_keep)
        print('-------')'''

'def create_directory_structure(dataset_dir:str, sub_dir:list, overwrite:bool):\n    if overwrite and os.path.exists(dataset_dir):\n        shutil.rmtree(dataset_dir)\n        \n    os.mkdir(dataset_dir)\n    \n    sub_dirs=[\'train\', \'validation\']\n    for sub_dir in sub_dirs:\n        inner_dir =f\'{dataset_dir}/{sub_dir}\' \n        os.mkdir(inner_dir)\n        os.mkdir(f\'{inner_dir}/{images_dir}\')\n        os.mkdir(f\'{inner_dir}/{images_resized_dir}\')\n        os.mkdir(f\'{inner_dir}/{bbox_dir}\')\n        os.mkdir(f\'{inner_dir}/{labels_dir}\')\n        os.mkdir(f\'{inner_dir}/{entities_dir}\')\n        os.mkdir(f\'{inner_dir}/{relations_dir}\')\n        os.mkdir(f\'{inner_dir}/{input_ids_dir}\')\n\ndef load_dataset_funsd(annotations_configs:dict=None, image_dirs:dict=None, dataset_dir:str= \'dataset_real\', tokenizer=None, model=None, train_test_split:float=0.05, overwrite :bool=True, dataset=\'all\'):\n    sub_dirs = [\'train\', \'validation\']\n    create_directory_struc

In [11]:
dataset='all'

In [12]:


root_dir = 'all_annotations/'
annotations_dirs = {dir:f'{root_dir}/{dir}/annotations/instances_default.json' for dir in os.listdir(root_dir)}
image_dirs = {dir:f'{root_dir}/{dir}/images' for dir in os.listdir(root_dir)}


load_dataset_funsd(annotations_configs=annotations_dirs,image_dirs=image_dirs, tokenizer=tokenizer, model=model)


populating valid image indices
populating valid image indices
populating valid image indices
populating valid image indices


In [21]:
split_data(train_test_split=0.05, dataset_dir='dataset_real')

[58, 163, 322, 18, 300, 3, 298, 49, 227, 302, 16, 121, 252, 182, 46, 314]


In [13]:
tokenizer_dir = 'tokenizer_added_tokens'
model_dir = 'model_added_tokens'
shutil.rmtree(tokenizer_dir)
shutil.rmtree(model_dir)
tokenizer.save_pretrained(tokenizer_dir)
model.save_pretrained(model_dir)

## Testing relations and entities

In [14]:
index = 200
entities_path = f'dataset_real/train/entities/entities_{index}.p'
with open(entities_path, 'rb') as f:
    entities = pickle.load(f)

relations_path = f'dataset_real/train/relations/relations_{index}.p'

with open(relations_path, 'rb') as f:
    relations = pickle.load(f)
    
input_ids_path = f'dataset_real/train/input_ids/input_ids_{index}.p'

with open(input_ids_path, 'rb') as f:
    input_ids = pickle.load(f)

#print(relations)
#print(entities)

entities_names = []
entities_with_boxes = []
for i , (start, end) in enumerate(zip(entities['start'], entities['end'])):
    #print(f'{i}, {tokenizer.decode(input_ids[start:end])} : {entities["label"][i]} {entities["cat"][i]}')
    #if entities["label"][i].lower() != 'other':
    entities_names.append(tokenizer.decode(input_ids[start:end]))
  #entities_names.append(tokenizer.decode(input_ids[start:end]))

for i in range(len(relations['head'])):
    print(f'question : {entities_names[relations["head"][i]]}, Answer :  {entities_names[relations["tail"][i]]}, start: {relations["head"][i]}, end: {relations["tail"][i]}')

question : equipment-nummer, Answer :  0010017431, start: 43, end: 44
question : auftrags-nummer, Answer :  8109589315-0110, start: 45, end: 46
question : kunden-nummer, Answer :  0055026449, start: 47, end: 48
question : akten-nummer, Answer :  88206-981764, start: 49, end: 50
question : leistungsort, Answer :  falkenried office 1strabenbahnring 3-520251hamburg, start: 51, end: 52
question : fabrik-/herstell-nr, Answer :  5291846, start: 53, end: 36
question : baujahr, Answer :  2003, start: 54, end: 0
question : tragfähigkeit, Answer :  1000 kg, start: 55, end: 39
question : hersteller, Answer :  konegmbh, start: 59, end: 37
question : ausführende fa, Answer :  konegmbh, start: 60, end: 38
question : prüfergebnis, Answer :  ohnemänge, start: 61, end: 40
question : prüfort, Answer :  hamburg, start: 62, end: 42
question : prüfdatum, Answer :  19.03.2013, start: 63, end: 41


## Testing bboxes without dataset class

In [15]:
index = 200

image = cv2.imread(f'dataset/train/images/image_{index}.jpeg')
bbox_path = f'dataset/train/bbox/bbox_{index}.p'
with open(bbox_path, 'rb') as f:
    bbox = pickle.load(f)

def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

image = Image.fromarray(image)
width, height = image.size
print(width)
print(height)

true_boxes = [unnormalize_box(box, width, height) for idx, box in enumerate(bbox)]
image = np.array(image)
for box in true_boxes:
  box = [int(i) for i in box]
  cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
plt.imshow(image)

[ WARN:0@765.774] global loadsave.cpp:244 findDecoder imread_('dataset/train/images/image_200.jpeg'): can't open/read file: check file path/integrity


FileNotFoundError: [Errno 2] No such file or directory: 'dataset/train/bbox/bbox_200.p'

## Testing bboxes

In [ ]:
from dataset_creator import *

configs_train = {
    "num_files":30,
    "images_dir" : "images",
    "images_resized_dir" : "images_resized",
    "bbox_dir":"bbox",
    "input_ids_dir" : "input_ids",
    "labels_dir" : "labels",
    "entities_dir" : "entities",
    "relations_dir" : "relations",
    "type":"train",
    "clear_all_old_files" : True,
    "clear_old_files_type": [True, "validation"]
}

configs_validation = {
    "num_files":30,
    "images_dir" : "images",
    "images_resized_dir" : "images_resized",
    "bbox_dir":"bbox",
    "input_ids_dir" : "input_ids",
    "labels_dir" : "labels",
    "entities_dir" : "entities",
    "relations_dir" : "relations",
    "type":"validation",
    "clear_all_old_files" : True,
    "clear_old_files_type": [True, "validation"]
}

configs_train = {key:val for key,val in configs_train.items() if key not in ("num_files", "clear_all_old_files", "clear_old_files_type")}
configs_validation = {key:val for key,val in configs_validation.items() if key not in ("num_files", "clear_all_old_files", "clear_old_files_type")}

dataset_train = Custom_Dataset(**configs_train)
dataset_validation = Custom_Dataset(**configs_validation)

In [ ]:
sample = dataset_train[1]
image = sample['original_image']
image.shape
bbox = sample['bbox']
bbox

In [ ]:
def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

#predictions = outputs.logits.argmax(-1).squeeze().tolist()
#token_boxes = encoding.bbox.squeeze().tolist()

image = Image.fromarray(image)
width, height = image.size
print(width)
print(height)

In [ ]:

from PIL import ImageDraw, ImageFont


draw = ImageDraw.Draw(image)

font = ImageFont.load_default()
true_boxes = [unnormalize_box(box, width, height) for idx, box in enumerate(bbox)]
#true_boxes = bbox
print(true_boxes)
def iob_to_label(label):
    label = label[2:]
    if not label:
      return 'other'
    return label

label2color = {'question':'blue', 'answer':'green', 'header':'orange', 'other':'violet'}
image = np.array(image)
for box in true_boxes:
  box = [int(i) for i in box]
  cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
    #draw.text((box[0]+10, box[1]-10), text=predicted_label, fill=label2color[predicted_label], font=font)



In [ ]:
plt.imshow(image)
